In [1]:
import numpy as np
from docplex.mp.model import Model

In [2]:
# Input data: Instance 07
n_dpts = 7
max_bays = np.arange(1, 10)
dpts = np.arange(1, n_dpts + 1)
width = 13
height = 8.54
dp_areas = np.array([16, 16, 16, 36, 9, 9, 9])
asp_ratio = np.array([5, 5, 5, 5, 5, 5, 5])
max_side = np.minimum(np.array([height]*n_dpts), np.sqrt(asp_ratio*dp_areas))
min_side = np.sqrt(dp_areas/asp_ratio)
mat_flows = np.array([[0,  0,  0,  0,  0,  0,  0],
                      [0,  0,  0,  0,  0,  0,  0],
                      [0,  0,  0,  0,  0,  0,  0],
                      [5,  3,  2,  0,  0,  0,  0],
                      [0,  0,  0,  4,  0,  0,  0],
                      [0,  0,  0,  4,  0,  0,  0],
                      [1,  1,  1,  0,  2,  1,  0]])

mat_flows = mat_flows.transpose()

In [3]:
# Define UAFLP-MIP model
uaflp_mip = Model('UAFLP-MIP-KONAK-2006')

# Add variables
zs = {(i, k): uaflp_mip.binary_var(name=f'z_{i}_{k}')\
      for i in dpts for k in max_bays}

rs = {(i, j): uaflp_mip.binary_var(name=f'r_{i}_{j}')\
      for i in dpts for j in dpts}

ds = {(k): uaflp_mip.binary_var(name=f'd_{k}')\
      for k in max_bays}

w = {(k): uaflp_mip.continuous_var(name=f'w_{k}', lb=0)\
      for k in max_bays}

ly = {(i): uaflp_mip.continuous_var(name=f'l_y_{i}', lb=0)\
      for i in dpts}

h = {(i, k): uaflp_mip.continuous_var(name=f'h_{i}_{k}', lb=0)\
       for i in dpts for k in max_bays}

x = {(i): uaflp_mip.continuous_var(name=f'x_{i}', lb=0)\
      for i in dpts}

y = {(i): uaflp_mip.continuous_var(name=f'y_{i}', lb=0)\
      for i in dpts}

dx = {(i, j): uaflp_mip.continuous_var(name=f'd_x_{i}_{j}', lb=0)\
        for i in dpts for j in dpts}

dy = {(i, j): uaflp_mip.continuous_var(name=f'd_y_{i}_{j}', lb=0)\
        for i in dpts for j in dpts}

In [4]:
# Add objective function
# total_mf_costs = uaflp_mip.linear_expr()

# for i in dpts:
#     for j in dpts:
#         if i < j:
#             print(i, j, mat_flows[i-1, j-1])
#             total_mf_costs.add_term(dx[i, j], mat_flows[i-1, j-1])
#             total_mf_costs.add_term(dy[i, j], mat_flows[i-1, j-1])


total_mf_costs = uaflp_mip.sum(uaflp_mip.sum(mat_flows[i-1, j-1] * (dx[i, j] + dy[i, j]) for j in dpts if i < j) for i in dpts)

uaflp_mip.minimize(total_mf_costs)

In [6]:
# Add constraints
for i in dpts:
    for j in dpts:
        if i < j:
            uaflp_mip.add_constraint(dx[i, j] >= x[i] - x[j])
            uaflp_mip.add_constraint(dx[i, j] >= x[j] - x[i])
            uaflp_mip.add_constraint(dy[i, j] >= y[i] - y[j])
            uaflp_mip.add_constraint(dy[i, j] >= y[j] - y[i])

In [7]:
for i in dpts:
    uaflp_mip.add_constraint(uaflp_mip.sum(zs[i, k] for k in max_bays) == 1)

In [8]:
for k in max_bays:
    uaflp_mip.add_constraint(w[k] == uaflp_mip.sum(zs[i, k] * dp_areas[i-1] for i in dpts)/height)

In [9]:
for i in dpts:
    for k in max_bays:
        uaflp_mip.add_constraint(min_side[i-1] * zs[i, k] <= w[k])
        uaflp_mip.add_constraint(w[k] <= max_side[i-1] + width * (1 - zs[i, k]))

In [10]:
for i in dpts:
    for k in max_bays:
        uaflp_mip.add_constraint(x[i] >= uaflp_mip.sum(w[j] for j in max_bays if j <= k) - 0.5*w[k] - \
                                 (width - min_side[i-1])*(1 - zs[i, k]))
        
        uaflp_mip.add_constraint(x[i] <= uaflp_mip.sum(w[j] for j in max_bays if j <= k) - 0.5*w[k] + \
                                 (width - min_side[i-1])*(1 - zs[i, k]))

In [11]:
for i in dpts:
    for j in dpts:
        if i < j:
            for k in max_bays:
                uaflp_mip.add_constraint(h[i, k]/dp_areas[i-1] - h[j, k]/dp_areas[j-1] - uaflp_mip.max([max_side[i-1]/dp_areas[i-1], max_side[j-1]/dp_areas[j-1]])*(2 - zs[i, k] - zs[j, k]) <= 0)
                uaflp_mip.add_constraint(h[i, k]/dp_areas[i-1] - h[j, k]/dp_areas[j-1] - uaflp_mip.max([max_side[i-1]/dp_areas[i-1], max_side[j-1]/dp_areas[j-1]])*(2 - zs[i, k] - zs[j, k]) >= 0)

In [12]:
for k in max_bays:
    uaflp_mip.add_constraint(uaflp_mip.sum(h[i, k] for i in dpts) == height*ds[k])

In [13]:
for i in dpts:
    for k in max_bays:
        uaflp_mip.add_constraint(min_side[i-1]*zs[i, k] <= h[i, k])
        uaflp_mip.add_constraint(h[i, k] <= max_side[i-1]*zs[i, k])

In [14]:
for i in dpts:
    uaflp_mip.add_constraint(uaflp_mip.sum(h[i, k] for k in max_bays) == ly[i])

In [15]:
for i in dpts:
    for j in dpts:
        if i != j:
            uaflp_mip.add_constraint(y[i] - 0.5*ly[i] >= y[j] + 0.5*ly[j] - height*(1 - rs[i, j]))

In [16]:
for i in dpts:
    for j in dpts:
        if i < j:
            uaflp_mip.add_constraint(rs[i, j] + rs[j, i] <= 1)

In [17]:
for i in dpts:
    for j in dpts:
        if i < j:
            for k in max_bays:
                uaflp_mip.add_constraint(rs[i, j] + rs[j, i] >= zs[i, k] + zs[j, k] - 1)

In [18]:
for i in dpts:
    uaflp_mip.add_constraint(0.5*ly[i] <= y[i])
    uaflp_mip.add_constraint(y[i] <= height - 0.5*ly[i])

In [ ]:
uaflp_mip.pprint()

In [19]:
uaflp_sol = uaflp_mip.solve(log_output=True)

CPLEX Error  1016: Community Edition. Problem size limits exceeded. Purchase at http://ibm.biz/error1016.


DOcplexLimitsExceeded: **** Promotional version. Problem size limits (1000 vars, 1000 consts) exceeded, model has 312 vars, 1138 consts, CPLEX code=1016